In [33]:
import os
import re
import json
import numpy as np
from dotenv import load_dotenv
from mistralai import Mistral
import faiss


In [34]:
# 0. Charger la clé depuis le fichier .env
load_dotenv()
api_key = os.getenv("MISTRAL_API_KEY")
client = Mistral(api_key=api_key)

In [38]:
# 1. Lire fichiers .py et .ipynb
def read_code(folder, extensions={".py", ".ipynb"}):
    code_files = []
    for dirpath, _, filenames in os.walk(folder):
        for f in filenames:
            ext = os.path.splitext(f)[1].lower()
            full_path = os.path.join(dirpath, f)
            if ext in extensions:
                if ext == ".ipynb":
                    with open(full_path, "r", encoding="utf-8") as file:
                        nb = json.load(file)
                        for cell in nb.get("cells", []):
                            if cell.get("cell_type") == "code":
                                code_files.append("".join(cell.get("source", [])))
                else:
                    with open(full_path, "r", encoding="utf-8") as file:
                        code_files.append(file.read())
    return code_files

# 2. Découper par fonction
def split_functions(code):
    pattern = r"(def [\w_]+\s*\(.*?\):(?:\n(?:\s+.+))*)"
    funcs = re.findall(pattern, code, re.DOTALL)
    return funcs if funcs else [code]

# 3. Créer embeddings
import numpy as np
import time
import random

def create_embeddings(chunks, batch_size=16, max_retries=5):
    """
    Crée des embeddings pour une liste de textes (chunks) en batch
    et gère les erreurs 429 avec retry exponentiel.
    
    Args:
        chunks (list of str): textes à encoder
        batch_size (int): nombre de chunks à envoyer par requête
        max_retries (int): nombre de tentatives en cas d'erreur 429
    Returns:
        embeddings (list of np.array): embeddings de chaque chunk
    """
    embeddings = []

    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i+batch_size]
        for attempt in range(max_retries):
            try:
                resp = client.embeddings.create(
                    model="codestral-embed",
                    inputs=batch
                )
                # Ajouter chaque embedding du batch
                for emb in resp.data:
                    embeddings.append(np.array(emb.embedding, dtype=np.float32))
                break  # sortir de la boucle retry si OK
            except Exception as e:
                # Vérifie si c'est une erreur 429
                if "429" in str(e):
                    wait = 2 ** attempt + random.random()
                    print(f"Erreur 429, tentative {attempt+1}/{max_retries}. Attente {wait:.2f}s")
                    time.sleep(wait)
                else:
                    raise e  # autre erreur → remonter
    return embeddings

# 4. Lire + chunker + embeddings
folder = "D:\\33611\\Documents\\MASTER\\M2\\BIG_DATA\\TP4\\data"
all_code = read_code(folder)
all_chunks = []
for code in all_code:
    all_chunks.extend(split_functions(code))
embeddings = create_embeddings(all_chunks,batch_size=8)
print(f"Nombre de chunks : {len(all_chunks)}")

# 5. Stocker dans FAISS
dim = len(embeddings[0])
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings))

# 6. Fonction pour poser une question au LLM avec RAG
def ask_question(query, top_k=3):
    """
    Pose une question sur le code en utilisant RAG avec Mistral plus léger.
    """
    # 1. Créer l'embedding de la question
    query_emb = client.embeddings.create(
        model="codestral-embed",  # embeddings restent les mêmes
        inputs=[query]
    )
    q_vec = np.array(query_emb.data[0].embedding, dtype=np.float32).reshape(1, -1)

    # 2. Chercher les chunks les plus proches dans FAISS
    D, I = index.search(q_vec, k=top_k)
    context = "\n\n".join([all_chunks[i] for i in I[0]])

    # 3. Préparer le prompt pour le LLM
    prompt = (
        "Voici le code extrait :\n"
        f"{context}\n\n"
        f"Question : {query}\n"
        "Réponds de manière claire et précise, explique le fonctionnement du code si nécessaire."
    )

    # 4. Appeler le LLM Mistral plus léger
    response = client.chat.complete(
        model="codestral-2508",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        max_tokens=600
    )

    return response.choices[0].message.content


# 7. Exemple
question = "A quoi sert le fichier generate_transaction.py ?"
answer = ask_question(question)
print("Réponse du LLM :\n", answer)


Nombre de chunks : 2
Réponse du LLM :
 # Fonction du fichier `generate_transaction.py`

Ce fichier Python a pour objectif de générer des fichiers CSV contenant des transactions bancaires de manière simulée, en imitant un flux continu de données.

## Fonctionnement détaillé du code :

1. **Importation des bibliothèques** :
   - `pandas` pour manipuler les données
   - `random` pour générer des nombres aléatoires
   - `os` pour gérer les chemins de fichiers
   - `time` pour gérer les temporisations et les horodatages

2. **Chargement des données** :
   - Le code lit un fichier CSV nommé `stream.csv` qui contient les transactions bancaires

3. **Boucle infinie** :
   - Le script entre dans une boucle infinie (`while True`) qui génère continuellement des fichiers de transactions

4. **Génération des sous-ensembles de transactions** :
   - À chaque itération, il sélectionne un nombre aléatoire de transactions (entre 1 et 150) à exporter
   - Il utilise un index `i` pour parcourir le fichier

In [36]:
print(client.models.list())



object='list' data=[BaseModelCard(id='mistral-medium-2505', capabilities=ModelCapabilities(completion_chat=True, completion_fim=False, function_calling=True, fine_tuning=True, vision=True, classification=False), object='model', created=1758788154, owned_by='mistralai', name='mistral-medium-2505', description='Our frontier-class multimodal model released May 2025.', max_context_length=131072, aliases=[], deprecation=None, deprecation_replacement_model=None, default_model_temperature=0.3, TYPE='base'), BaseModelCard(id='mistral-large-latest', capabilities=ModelCapabilities(completion_chat=True, completion_fim=False, function_calling=True, fine_tuning=True, vision=True, classification=False), object='model', created=1758788154, owned_by='mistralai', name='mistral-large-latest', description='Official mistral-large-latest Mistral AI model', max_context_length=131072, aliases=[], deprecation=None, deprecation_replacement_model=None, default_model_temperature=0.3, TYPE='base'), BaseModelCard(